In [25]:
!pip install natsort
import natsort

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from datamanip import DatasetLoader
from imutils import paths
import numpy as np
from sklearn.cluster import KMeans
import copy
from sklearn.svm import LinearSVC

In [4]:
# import the necessary packages
import numpy as np
import os
import cv2

class ImagePreprocessor:
    # For resizing image
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# Store the image width, height, and interpolation
		self.width = width
		self.height = height
		self.inter = inter
	def preprocess(self, image):
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

class DatasetLoader:
    def __init__(self, preprocessors=None):
        # Store the image preprocessor
        self.preprocessors = preprocessors
        # if there are no preprocessors, initialize them as an empty list
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, x=-1):
        # initialize the data and labels
        data = []
        labels = []
        # loop over all the images
        for (i, imagePath) in enumerate(imagePaths):
            #Load the image in grayscale mode
            image = cv2.imread(imagePath, 0)
            label = imagePath.split(os.path.sep)[-2]

            # if there are preprocessors
            if self.preprocessors is not None:
                # loop over the preprocessors and apply them to the image
                for p in self.preprocessors:
                    image = p.preprocess(image)
            # Append the images and labels
            data.append(image)
            labels.append(label)

            # Display the update every `x` images
            if x > 0 and i > 0 and (i + 1) % x == 0:
                print("[INFO] processed {}/{}".format(i + 1, len(imagePaths)))

        # return the images and labels
        return (np.array(data), np.array(labels))

In [28]:
# grab the list of images that we'll be describing
print("Loading Images")
imagePaths = list(paths.list_images('E:/Sem2/ComputerVision/Courseworks/Coursework3/training'))
testPaths = list(paths.list_images('E:/Sem2/ComputerVision/Courseworks/Coursework3/testing'))
testPaths = (natsort.natsorted(testPaths,reverse=False))
for i in testPaths:
    print(i)
# initialize the image preprocessor, load the dataset from disk,and reshape the data matrix
# sp = ImagePreprocessor(100, 100)
# dataset = SimpleDatasetLoader(preprocessors=[sp])
dataset = DatasetLoader()
(data, olabels) = dataset.load(imagePaths, x=500)
(testX, tlabels) = dataset.load(testPaths, x=500)
print("Train Data Shape = ", data.shape)
print("Test Data Shape = ", testX.shape)

Loading Images
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\0.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\1.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\2.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\3.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\4.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\5.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\6.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\7.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\8.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\9.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\10.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\11.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\12.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\13.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\14.jpg
E:/Sem2/ComputerVision/Courseworks/Coursework3/testing\15.jpg
E:/

[INFO] processed 500/1500
[INFO] processed 1000/1500
[INFO] processed 1500/1500


<ipython-input-4-8d1cded36e12>:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (np.array(data), np.array(labels))


[INFO] processed 500/2985
[INFO] processed 1000/2985
[INFO] processed 1500/2985
[INFO] processed 2000/2985
[INFO] processed 2500/2985
Train Data Shape =  (1500,)
Test Data Shape =  (2985,)


In [7]:
print("Labels before encoding", olabels)
# encode the labels as integers
label_encode = LabelEncoder()
labels = label_encode.fit_transform(olabels)
print("Labels after encoding", labels)
#Split the Training set into Training set and Validation set
(trainX, validX, trainY, validY) = train_test_split(data, labels, test_size=0.10, random_state=42)


Labels before encoding ['bedroom' 'bedroom' 'bedroom' ... 'TallBuilding' 'TallBuilding'
 'TallBuilding']
Labels after encoding [10 10 10 ...  9  9  9]


In [8]:
M=32 #Window Width
N=32 #Window Height
step = 16
descount = 0
tdescount = 0
desc_list = []
print("Preparing Input data for KMeans Clustering")
#Train Images Descriptors
for i in range(len(trainX)):
    train_flat_img = np.empty((1, int(M*N)))
    img = trainX[i]
    num_patches = 0
    endX = ((img.shape[0] - M) // step) * step #Last pixel row-wise to step
    endY = ((img.shape[1] - N) // step) * step #Last pixel column-wise to step
    #Stores all the patches in the images in a list
    patches = [img[x:x + M, y:y + N] for x in range(0, endX, step) for y in range(0, endY, step)]
    train_flat_img[0] = patches[0].flatten()
    for subimg in patches[1:]:
        train_flat_img = np.vstack((train_flat_img,subimg.flatten()))
        descount += 1
        num_patches+=1
    desc_list.append(train_flat_img)

#Stack all the patches together to be passed as input to KMeans Clustering Algorithm
all_train_desc = []
for i in range(len(desc_list)):
    for j in range(desc_list[i].shape[0]):
        all_train_desc.append(desc_list[i][j,:])

all_train_desc = np.array(all_train_desc)
print("All desc", all_train_desc)
print("All desc Shape", all_train_desc.shape)
print("Finisehd preparing the input data for KMeans Clustering")

Preparing Input data for KMeans Clustering
All desc [[ 44.  62. 128. ...  37.  42.  55.]
 [ 61.  75. 132. ...  49.  65.  75.]
 [ 27.  21.  33. ...  43.  40.  50.]
 ...
 [ 73.  82.  86. ...  82.  80.  81.]
 [117. 132. 124. ...  96.  92.  99.]
 [ 28.  29.  37. ... 108. 102. 100.]]
All desc Shape (256911, 1024)
Finisehd preparing the input data for KMeans Clustering


In [9]:
#validation Images Descriptors
v_desc_list = []
print("Getting the Validation Image Patches")
for i in range(len(validX)):
    valid_flat_img = np.empty((1, int(M*N)))
    img = validX[i]
    num_patches = 0
    endX = ((img.shape[0] - M) // step) * step #Last pixel row-wise to step
    endY = ((img.shape[1] - N) // step) * step #Last pixel column-wise to step
    # Stores all the patches in the images in a list
    patches = [img[x:x + M, y:y + N] for x in range(0, endX, step) for y in range(0, endY, step)]
    valid_flat_img[0] = patches[0].flatten()
    for subimg in patches[1:]:
        valid_flat_img = np.vstack((valid_flat_img,subimg.flatten()))
        descount += 1
        num_patches+=1
    v_desc_list.append(valid_flat_img)
print("Completed getting the Validation Image Patches")

Getting the Test Image Patches
Completed getting the Validation Image Patches


In [29]:
#Test Images Descriptors
t_desc_list = []
print("Getting the Test Image Patches")
for i in range(len(testX)):
    test_flat_img = np.empty((1, int(M*N)))
    img = testX[i]
    num_patches = 0
    endX = ((img.shape[0] - M) // step) * step #Last pixel row-wise to step
    endY = ((img.shape[1] - N) // step) * step #Last pixel column-wise to step
    # Stores all the patches in the images in a list
    patches = [img[x:x + M, y:y + N] for x in range(0, endX, step) for y in range(0, endY, step)]
    test_flat_img[0] = patches[0].flatten()
    for subimg in patches[1:]:
        test_flat_img = np.vstack((test_flat_img,subimg.flatten()))
        descount += 1
        num_patches+=1
    t_desc_list.append(test_flat_img)
print("Completed getting the Test Image Patches")

Getting the Test Image Patches
Completed getting the Test Image Patches


In [12]:
# build BoW representation from patches of training images using KMeans
def clustering(train_desc, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(train_desc)
    return kmeans


# form training set histograms for each training image using BoW representation
def createHistogram(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1, -1).ravel())

    return np.array(train_hist)

In [13]:
k = 80
print("KMeans Clustering Started")
kmeans = clustering(all_train_desc, k)
print("KMeans Clustering Done")
print("K Means Centers", kmeans.cluster_centers_)
train_hist = createHistogram(desc_list, kmeans, k)
valid_hist = createHistogram(v_desc_list, kmeans, k)
test_hist = createHistogram(t_desc_list, kmeans, k)

KMeans Clustering Started
KMeans Clustering Done
K Means Centers [[ 80.8143483   80.2354874   80.3033954  ...  71.67086528  72.91401972
   74.25903614]
 [138.78149606 139.19750656 138.82020997 ... 196.65485564 196.81824147
  195.56430446]
 [ 88.44307832  87.86156648  87.32832423 ...  61.11566485  61.85018215
   63.69535519]
 ...
 [ 91.91128609  91.79265092  92.16902887 ...  48.96167979  49.79685039
   50.12020997]
 [106.62064965 106.54021655 107.16976025 ... 156.99342614 156.2575406
  154.69102862]
 [ 88.03612663  87.29273743  87.58435754 ... 137.33221601 136.76759777
  135.26666667]]


In [14]:
# normalize Training histograms
# preprocess Training histograms
scale_values = preprocessing.StandardScaler().fit(train_hist)
train_hist = scale_values.transform(train_hist)

# normalize Validation histograms
# preprocess Validation histograms
scale_values = preprocessing.StandardScaler().fit(valid_hist)
valid_hist = scale_values.transform(valid_hist)

# normalize Testing histograms
# preprocess Testing histograms
scale_values = preprocessing.StandardScaler().fit(test_hist)
test_hist = scale_values.transform(test_hist)

In [61]:
print(test_hist.shape)

(2985, 80)


In [74]:
print("Validation Labels: ", validY) 
print("SVC Training")
# y_traing = np.zeros((len(trainY), 1))
# print(y_traing.shape)
# y = copy.deepcopy(y_traing)
# y_predict = np.zeros((len(test_hist), 1))
# for i in range(len(test_hist)):  # 1 vs 15 SVM
#     index = 0
#     test = np.array([test_hist[i,:]]).T
#     for j in range(len(labelnames)): # One hot encoding and training classifier for each class 
#         y = copy.deepcopy(y_traing)
#         y[index:index+90, 0:1] = np.ones((90,1))
#         clf = LinearSVC(random_state=0, C=0.06148)
#         clf.fit(train_hist, y.ravel())
#         if j == 0:
#             maxScore = np.dot(clf.coef_, test) + clf.intercept_
#             y_predict[i, 0:1] = j
#         elif np.dot(clf.coef_, test) + clf.intercept_ > maxScore:
#             maxScore = np.dot(clf.coef_, test) + clf.intercept_
#             y_predict[i, 0:1] = j
#         index = index + 90
# print(y_predict)
clf = LinearSVC(random_state=0, max_iter= 10000)
clf.fit(train_hist, trainY)
predict = clf.predict(valid_hist)
print("Accuracy:", np.mean(predict == validY)*100, "%")
print("Predictions:", predict)


Validation Labels:  [14  8 11 11 11  4  6 13  7  1  4  9 13  8 10  7  9 12 11 10 14 14 14  7
  9 11  9  9  7  7  6 12  7  6  1  2 11 11 14 10  8 11  8  4  6  2  2  2
 13  5 12  3 14  7  3 12 10 11  2  5 12  3 11 10  4 11  2  6  7  0  2  4
 13  1 13  8 12  3 11  5  6 10  1 13  4 10 12  5 13  5 14  1  1  9  6 11
 11  0 14  2  5 13 14  6 11 10 10 14 10  4 11  8 13  1  0 11  4  0  0  6
  5  8  9 11  0  7  7 13  7 11  7 10  6  9  3 10 11  2  5  1 14  1  9  1
  3  8 12  6  2 13]
SVC Training
Accuracy: 41.333333333333336 %
Predictions: [12  6  8  6 14  1  0  3  7  4  9  9  5  8 11  7  9  5  8  1  1 14  9  7
 10  8 11  9 14  7  3  7  1  6  1  2  0  8 14  1  3  8  8 11  2  2  8  2
  8  5 12  6 12  4 11 10  1  6  2  0 12  6 10 11  0 11  2  2  7  4  2  4
 11  1 10  8 13  3 10  5  1 10 11 13  0 10 10  5  1  9 12 14  1  9  6 11
  5  0  8  3  5 12 14  0  9  4  0  1 10  4 13  8  4  1 11  9  4  2  0  6
  5  8  7 14  0  7  5 13  7 11  1 10  0  1  9 13  3  2  1  3 14  1  9  1
  6  8  0  0  2  3]


In [75]:
actual_labels = ['bedroom', 'Coast', 'Forest', 'Highway', 'industrial', 'Insidecity', 'kitchen', 'livingroom', 'Mountain', 'Office', 'OpenCountry', 'store', 'Street', 'Suburb', 'TallBuilding']
from os import walk
filenames = next(walk('E:/Sem2/ComputerVision/Courseworks/Coursework3/testing'), (None, None, []))[2]  # [] if no file
filenames = (natsort.natsorted(filenames,reverse=False))
for i in range(len(filenames)):
    predict = clf.predict(test_hist[i].reshape(1, -1))
    print(filenames[i], actual_labels[predict[0]] )

0.jpg livingroom
1.jpg TallBuilding
2.jpg kitchen
3.jpg Forest
4.jpg OpenCountry
5.jpg Forest
6.jpg Insidecity
7.jpg Street
8.jpg Coast
9.jpg bedroom
10.jpg Office
11.jpg OpenCountry
12.jpg OpenCountry
13.jpg kitchen
14.jpg Highway
15.jpg Highway
16.jpg Street
17.jpg TallBuilding
18.jpg Suburb
19.jpg Mountain
20.jpg bedroom
21.jpg kitchen
22.jpg OpenCountry
23.jpg Insidecity
24.jpg bedroom
25.jpg Mountain
26.jpg Forest
27.jpg industrial
28.jpg Insidecity
29.jpg Forest
30.jpg kitchen
31.jpg TallBuilding
32.jpg livingroom
33.jpg Office
34.jpg industrial
35.jpg TallBuilding
36.jpg bedroom
37.jpg kitchen
38.jpg kitchen
39.jpg Forest
40.jpg TallBuilding
41.jpg Insidecity
42.jpg livingroom
43.jpg Highway
44.jpg livingroom
45.jpg bedroom
46.jpg Forest
47.jpg Insidecity
48.jpg Highway
49.jpg bedroom
50.jpg Forest
51.jpg TallBuilding
52.jpg Street
53.jpg industrial
54.jpg livingroom
55.jpg Highway
56.jpg Office
57.jpg Insidecity
58.jpg Forest
59.jpg livingroom
60.jpg Office
61.jpg Coast
62.jpg 

1322.jpg Insidecity
1323.jpg TallBuilding
1324.jpg Street
1325.jpg bedroom
1326.jpg bedroom
1327.jpg TallBuilding
1328.jpg Forest
1329.jpg kitchen
1330.jpg industrial
1331.jpg Insidecity
1332.jpg TallBuilding
1333.jpg industrial
1334.jpg Forest
1335.jpg bedroom
1336.jpg Coast
1337.jpg Forest
1338.jpg Highway
1339.jpg OpenCountry
1340.jpg kitchen
1341.jpg Coast
1342.jpg Street
1343.jpg Street
1344.jpg Coast
1345.jpg livingroom
1346.jpg bedroom
1347.jpg livingroom
1348.jpg OpenCountry
1349.jpg Forest
1350.jpg TallBuilding
1351.jpg Suburb
1352.jpg Suburb
1353.jpg Forest
1354.jpg bedroom
1355.jpg store
1356.jpg Highway
1357.jpg Mountain
1358.jpg Highway
1359.jpg TallBuilding
1360.jpg bedroom
1361.jpg TallBuilding
1362.jpg Coast
1363.jpg store
1364.jpg kitchen
1365.jpg Street
1366.jpg Coast
1367.jpg OpenCountry
1368.jpg TallBuilding
1369.jpg bedroom
1370.jpg Forest
1371.jpg kitchen
1372.jpg livingroom
1373.jpg kitchen
1374.jpg Office
1375.jpg Forest
1376.jpg Street
1377.jpg TallBuilding
137

2572.jpg Mountain
2573.jpg Office
2574.jpg Coast
2575.jpg Coast
2576.jpg TallBuilding
2577.jpg Coast
2578.jpg kitchen
2579.jpg Mountain
2580.jpg livingroom
2581.jpg OpenCountry
2582.jpg kitchen
2583.jpg Forest
2584.jpg Insidecity
2585.jpg Forest
2586.jpg Highway
2587.jpg bedroom
2588.jpg kitchen
2589.jpg Forest
2590.jpg Highway
2591.jpg kitchen
2592.jpg Mountain
2593.jpg OpenCountry
2594.jpg Insidecity
2595.jpg TallBuilding
2596.jpg Street
2597.jpg OpenCountry
2598.jpg Coast
2599.jpg kitchen
2600.jpg Insidecity
2601.jpg OpenCountry
2602.jpg Suburb
2603.jpg kitchen
2604.jpg bedroom
2605.jpg Highway
2606.jpg bedroom
2607.jpg kitchen
2608.jpg Coast
2609.jpg Mountain
2610.jpg Forest
2611.jpg TallBuilding
2612.jpg bedroom
2613.jpg TallBuilding
2614.jpg industrial
2615.jpg Highway
2616.jpg Insidecity
2617.jpg Office
2618.jpg Mountain
2619.jpg bedroom
2620.jpg Office
2621.jpg Office
2622.jpg Office
2623.jpg Street
2624.jpg kitchen
2625.jpg Mountain
2626.jpg Office
2627.jpg OpenCountry
2628.jp

[7]
